In [1]:
'''
Example: Deterministic record linkage to find links in a single file.

In determininistic record linkage, each compared attribute get a certain
weight (coefficient). The higher the weight, the more dicriminating the
variable is. A low weight indicate a less discriminating variable. For
example, the given name has a higher weight than the hometown.

This example uses FEBRL3 datasets. This dataset contain records about
individuals.

Deterministic RL parameters are:
intercept = -11.0
coefficients = [1.5, 1.5, 8.0, 6.0, 2.5, 6.5, 5.0]
'''

'Example: Deterministic record linkage to find links in a single file.\n\nIn determininistic record linkage, each compared attribute get a certain\nweight (coefficient). The higher the weight, the more dicriminating the\nvariable is. A low weight indicate a less discriminating variable. For\nexample, the given name has a higher weight than the hometown.\n\nThis example uses FEBRL3 datasets. This dataset contain records about\nindividuals.\n\nDeterministic RL parameters are:\nintercept = -11.0\ncoefficients = [1.5, 1.5, 8.0, 6.0, 2.5, 6.5, 5.0]\n\n'

In [2]:
from __future__ import print_function

import recordlinkage as rl
from recordlinkage.index import Block
from recordlinkage.compare import Exact, String

In [ ]:
from recordlinkage.datasets import load_febrl3

'''
This data set contains 5000 records (2000 originals and 3000 duplicates), 
with a maximum of 5 duplicates based on one original record (and a Zipf distribution of duplicate records). 
Distribution of duplicates: 
    168 originals records have 5 duplicate records 
    161 originals records have 4 duplicate records 
    212 originals records have 3 duplicate records 
    256 originals records have 2 duplicate records 
    368 originals records have 1 duplicate record 1
    835 originals records have no duplicate record   
'''

In [3]:
# set logging
rl.logging.set_verbosity(rl.logging.INFO)

In [4]:
# load dataset
print('Loading data...')
dfA, true_links = load_febrl3(return_links=True)
print(len(dfA), 'records in dataset A')
print(len(true_links), 'links in dataset A')

Loading data...
5000 records in dataset A
6538 links in dataset A


In [7]:
# start indexing
print('Build index...')
indexer = rl.Index()
indexer.add(Block('given_name'))
indexer.add(Block('surname'))
indexer.add(Block('soc_sec_id'))
candidate_links = indexer.index(dfA)
print(len(candidate_links), 'candidate links in dataset A')

Build index...
INFO:recordlinkage:indexing - initialize Index class
INFO:recordlinkage:indexing [1/?] - time: 0.40s - pairs: 75984/12497500 - rr: 0.99392
INFO:recordlinkage:indexing [1/?] - time: 0.40s - pairs_total: 75984/12497500 - rr_total: 0.99392
75984 candidate links in dataset A


In [8]:
# start comparing
print('Start comparing...')
comparer = rl.Compare()
comparer.add(Exact('given_name', 'given_name', label='given_name'))
comparer.add(String('surname', 'surname', method='jarowinkler',
                    threshold=0.85, label='surname'))
comparer.add(Exact('date_of_birth', 'date_of_birth', label='date_of_birth'))
comparer.add(Exact('suburb', 'suburb', label='suburb'))
comparer.add(Exact('state', 'state', label='state'))
comparer.add(String('address_1', 'address_1', threshold=0.85,
                    label='address_1'))
comparer.add(String('address_2', 'address_2', threshold=0.85,
                    label='address_2'))
features = comparer.compute(candidate_links, dfA)

print('feature shape', features.shape)

Start comparing...
INFO:recordlinkage:comparing - initialize Compare class
INFO:recordlinkage:comparing [1/?] - time: 1.48s - pairs: 75984
INFO:recordlinkage:comparing [1/?] - time: 1.48s - pairs_total: 75984
feature shape (75984, 7)


In [9]:
# use the Logistic Regression Classifier
# this classifier is equivalent to the deterministic record linkage approach
intercept = -9.5
coefficients = [2.0, 3.0, 7.0, 6.0, 2.5, 5.0, 5.5]

print('Deterministic classifier')
print('intercept', intercept)
print('coefficients', coefficients)

logreg = rl.LogisticRegressionClassifier(
    coefficients=coefficients, intercept=intercept)
links = logreg.predict(features)

print(len(links), 'links/matches')

Deterministic classifier
intercept -9.5
coefficients [2.0, 3.0, 7.0, 6.0, 2.5, 5.0, 5.5]
INFO:recordlinkage:Classification - predict matches and non-matches
6224 links/matches


In [10]:
# return the confusion matrix
conf_logreg = rl.confusion_matrix(true_links, links, len(candidate_links))
print('confusion matrix')
print(conf_logreg)

confusion matrix
[[ 6206   332]
 [   18 69428]]


In [11]:
# compute the F-score for this classification
fscore = rl.fscore(conf_logreg)
print('fscore', fscore)
recall = rl.recall(true_links, links)
print('recall', recall)
precision = rl.precision(true_links, links)
print('precision', precision)

fscore 0.972574831531108
recall 0.9492199449372897
precision 0.997107969151671
